## Part 1: Preprocessing

In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [8]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [9]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [35]:
y_df['Attrition'].value_counts()

,count
Attrition,
No,1233
Yes,237


In [36]:
y_df['Department'].value_counts()

,count
Department,
Research & Development,961
Sales,446
Human Resources,63


In [10]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
             'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [11]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df['OverTime'] = X_df['OverTime'].fillna(0).replace({'No': 0, 'Yes': 1}).astype(int)
X_df.dtypes

<ipython-input-11-167c49f95138>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].fillna(0).replace({'No': 0, 'Yes': 1}).astype(int)


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [12]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [14]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [15]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])


# Create two new variables for the encoded data
y_train_encoded_dept = dept_encoder.transform(y_train[['Department']])
y_test_encoded_dept = dept_encoder.transform(y_test[['Department']])


In [16]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_attr = attr_encoder.transform(y_train[['Attrition']])
y_test_encoded_attr = attr_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [18]:
# Find the number of columns in the X training data
number_input_features = X_train_scaled.shape[1]

# Create the input layer
input = layers.Input(shape=(number_input_features,), name='input_features')

# Create at least two shared layers
hidden1 = layers.Dense(64, activation='relu')(input)
hidden2 = layers.Dense(32, activation='relu')(hidden1)

In [26]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(3, activation='relu')(hidden2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [27]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(2, activation='relu')(hidden2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [28]:
# Create the model

model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 3)              │             99 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 2)              │             66 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             12 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │              6 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,967 (11.59 KB)

 Trainable params: 2,967 (11.59 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': y_train_encoded_dept, 'attrition_output': y_train_encoded_attr},
     epochs=100,
     batch_size=32,
     validation_split=0.2
)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - attrition_output_accuracy: 0.8520 - department_output_accuracy: 0.5918 - loss: 1.7380 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.6640
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8449 - department_output_accuracy: 0.6758 - loss: 1.6051 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.6098
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8408 - department_output_accuracy: 0.6429 - loss: 1.5727 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.4948
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8337 - department_output_accuracy: 0.6762 - loss: 1.3835 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.4005
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6

In [30]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test_encoded_dept, 'attrition_output': y_test_encoded_attr})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8480 - department_output_accuracy: 0.5709 - loss: 1.8910 


[1.9167664051055908, 0.8478260636329651, 0.5652173757553101]

In [34]:
# Print the accuracy for both department and attrition
department_accuracy = test_results[2]
attrition_accuracy = test_results[1]
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

Department Accuracy: 0.5652173757553101
Attrition Accuracy: 0.8478260636329651


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy is not the best metric to use because the attrition data is highly imbalanced (1233 'No' to 237 'Yes'). A model could achieve high accuracy by simply predicting "No" most of the time, which wouldn't necessarily mean it's good at identifying employees who are likely to leave.
2. For the attrition output, I used the sigmoid activation function. This is best for for when there are only 2 outputs like 'Yes' and 'No'. I used softmax activation function for the department output since there were more than 2 outputs.
3. Given the imbalance in the attrition data I would consider addressing it by using techniques like oversampling the minority class ('Yes') and undersampling the majority class ('No'). Besides neural networks, I might also try a model like Random Forest, which usually performs well on imbalanced data and can provide better interpretability.